**Authors**

Umberto Altieri - umberto.altieri@studio.unibo.it 



#Import libraries

In [ ]:
!pip install transformers
import pandas as pd
import random
import tensorflow as tf
import numpy as np
import torch
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import string
import nltk
import numpy as np
import csv
import os
from nltk.stem import 	WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score
import pickle
from transformers import *
from tqdm import tqdm, trange
from ast import literal_eval
import urllib.request
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast as BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import copy
wordnet_lemmatizer = WordNetLemmatizer()
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


#The GPU used is Tesla T4

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

#Reproducibility

In [ ]:
def set_reproducibility(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

set_reproducibility(42)

#Import data

The data are downloaded by the links provided by the challenge website. On the  [website](https://zenodo.org/record/7503506) are provided: 1 dataset for the training, 2 for the validation and 1 for the test set. Unfortunately this last one is without labels, as the evaluation of the performance on the test dataset is provided through the website tira, where the labels produced by the model should be uploaded to know the performances of the problem, that will be evaluated at the end of the challenge.
It is for this reason that the model will be evaluated on one of the two validation dataset. 

In [ ]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)
        
def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

def download_data(data_path, url_path, suffix):    
    if not os.path.exists(data_path):
        os.makedirs(data_path)
        
    data_path = os.path.join(data_path, f'{suffix}.tsv')
  
    if not os.path.exists(data_path):
        print(f"Downloading dataset {suffix} ... (it may take a while)")
        download_url(url=url_path, output_path=data_path)
        print("Download completed!")

In [ ]:
# Train data examples
train_url = "https://zenodo.org/record/7402070/files/arguments-training.tsv?download=1"
download_data(data_path='data', url_path = train_url, suffix='train_examples')

# Train data labels
train_url = "https://zenodo.org/record/7402070/files/labels-training.tsv?download=1"
download_data(data_path='data', url_path = train_url, suffix='train_labels')

# Validation data examples
train_url = "https://zenodo.org/record/7402070/files/arguments-validation.tsv?download=1"
download_data(data_path='data', url_path = train_url, suffix='validation_examples')

# Validation data labels
train_url = "https://zenodo.org/record/7402070/files/labels-validation.tsv?download=1"
download_data(data_path='data', url_path = train_url, suffix='validation_labels')

# Test data
test_url = "https://zenodo.org/record/7402070/files/arguments-test.tsv?download=1"
download_data(data_path='data', url_path=test_url, suffix='test') 


In [ ]:
# Train data arguments
train_url_arguments = "https://zenodo.org/record/7402070/files/arguments-training.tsv?download=1"
download_data(data_path='data', url_path = train_url_arguments, suffix='train_arguments')
# Train data labels
train_url_labels = "https://zenodo.org/record/7402070/files/labels-training.tsv?download=1"
download_data(data_path='data', url_path = train_url_labels, suffix='train_labels')




# Validation data arguments
val_url_arguments = "https://zenodo.org/record/7402070/files/arguments-validation.tsv?download=1"
download_data(data_path='data', url_path = val_url_arguments, suffix='validation_arguments')
# Validation data labels
val_url_labels = "https://zenodo.org/record/7402070/files/labels-validation.tsv?download=1"
download_data(data_path='data', url_path = val_url_labels, suffix='validation_labels')



# Validation data arguments zhihu
val_url_arguments_zhihu = "https://zenodo.org/record/7402070/files/arguments-validation-zhihu.tsv?download=1"
download_data(data_path='data', url_path = val_url_arguments_zhihu, suffix='validation_arguments_zhihu')
# Validation data labels zhihu
val_url_arguments_zhihu = "https://zenodo.org/record/7402070/files/labels-validation-zhihu.tsv?download=1"
download_data(data_path='data', url_path = val_url_arguments_zhihu, suffix='validation_labels_zhihu')



The values in the data are separated by the /t character, as they are .tsv files, loaded as pandas dataframes.

In [ ]:

tsv_train_arguments_path='/content/data/train_arguments.tsv'
tsv_validation_arguments_path='/content/data/validation_arguments.tsv'
tsv_validation_arguments_zhihu_path='/content/data/validation_arguments_zhihu.tsv'



tsv_train_labels_path='/content/data/train_labels.tsv'
tsv_validation_labels_path='/content/data/validation_labels.tsv'
tsv_validation_labels_zhihu_path='/content/data/validation_labels_zhihu.tsv'


tsv_train_arguments = pd.read_csv(tsv_train_arguments_path, sep='\t')
tsv_validation_arguments = pd.read_csv(tsv_validation_arguments_path, sep='\t')
tsv_validation_arguments_zhihu = pd.read_csv(tsv_validation_arguments_zhihu_path, sep='\t')

tsv_train_labels = pd.read_csv(tsv_train_labels_path, sep='\t')
tsv_validation_labels = pd.read_csv(tsv_validation_labels_path, sep='\t')
tsv_validation_labels_zhihu = pd.read_csv(tsv_validation_labels_zhihu_path, sep='\t')




Creation of the dataframes, the training dataset is merged with the first validation one. While the second validation dataset, called zhihu, is used to test the model.



In [ ]:
df_train= pd.concat([tsv_train_labels,tsv_train_arguments],axis=1)
df_val=pd.concat([tsv_validation_labels,tsv_validation_arguments],axis=1)
df_val_zhihu=pd.concat([tsv_validation_labels_zhihu,tsv_validation_arguments_zhihu],axis=1)

In [ ]:
label_cols = list(df_train.columns[0:-3])

In [ ]:
print('Train Count of 1 per label: \n', df_train[label_cols].sum(), '\n')  
print('Train Count of 0 per label: \n', df_train[label_cols].eq(0).sum())

Train Count of 1 per label: 
 Argument ID                   A01002A01005A01006A01007A01008A01009A01010A010...
Argument ID                   A01002A01005A01006A01007A01008A01009A01010A010...
Self-direction: thought                                                     988
Self-direction: action                                                     1395
Stimulation                                                                 247
Hedonism                                                                    172
Achievement                                                                1512
Power: dominance                                                            610
Power: resources                                                            625
Face                                                                        382
Security: personal                                                         2000
Security: societal                                                         1728
Tradition 

In [ ]:
df_train.shape

(5393, 25)

In [ ]:
print('Validation Count of 1 per label: \n', df_val_zhihu[label_cols].sum(), '\n')  
print('Validation Count of 0 per label: \n', df_val_zhihu[label_cols].eq(0).sum())

Validation Count of 1 per label: 
 Argument ID                   C26001C26002C26003C26004C26005C26006C26007C260...
Argument ID                   C26001C26002C26003C26004C26005C26006C26007C260...
Self-direction: thought                                                       6
Self-direction: action                                                       11
Stimulation                                                                   0
Hedonism                                                                      2
Achievement                                                                  39
Power: dominance                                                              1
Power: resources                                                             19
Face                                                                          1
Security: personal                                                           30
Security: societal                                                           31
Tradi

In [ ]:
df_val_zhihu.shape

(100, 25)

In [ ]:
print('Validation Count of 1 per label: \n', df_val[label_cols].sum(), '\n')  
print('Validation Count of 0 per label: \n', df_val[label_cols].eq(0).sum())

Validation Count of 1 per label: 
 Argument ID                   A01001A01012A02001A02002A02009A02018A03005A030...
Argument ID                   A01001A01012A02001A02002A02009A02018A03005A030...
Self-direction: thought                                                     251
Self-direction: action                                                      496
Stimulation                                                                 138
Hedonism                                                                    103
Achievement                                                                 575
Power: dominance                                                            164
Power: resources                                                            132
Face                                                                        130
Security: personal                                                          759
Security: societal                                                          488
Tradi

In [ ]:
df_val.shape

(1896, 25)

It can be seen from the labels count that the dataset of training, validation and validation zhihu keep the same proprotion with respect to their size, but no one of them is balanced, as the difference in number of occurrences between labels is often very large. 

For this reason, as suggested by the Touchè company I took into account only six labels: Self-direction: action, Achievement, Security: personal, Security: societal, Benevolence: caring, Universalism: concern.
Considering only the mentioned classes, the dataset becomes more balanced. In addition, I have added the validation dataset to the training one in order to enlarge the training set, which otherwise would have too few samples.

In [ ]:
df_train=pd.concat([df_train,df_val],ignore_index=True,axis=0)
df_val=df_val_zhihu

Dropping not needed columns

In [ ]:
del df_train['Argument ID']
del df_val['Argument ID']


I narrow down the dataset according to the 6 labels : Self-direction: action, Achievement, Security: personal, Security: societal, Benevolence: caring, Universalism: concern.


In [ ]:

label_cols=['Self-direction: action', 'Achievement', 'Security: personal', 'Security: societal', 'Benevolence: caring', 'Universalism: concern']
num_labels=len(label_cols)
all_labels=copy.copy(label_cols)
all_labels.append('Premise')
all_labels.append('Stance')
all_labels.append('Conclusion')

In [ ]:
df_train=df_train[all_labels]
df_val=df_val[all_labels]
  

Drop the rows in the dataset that do not belong to any of the selected six classes.

In [ ]:
df_train = df_train.loc[(df_train[label_cols] != 0).any(axis=1)]

df_val= df_val.loc[(df_val[label_cols] != 0).any(axis=1)]

The shape of the 2 datasets shrinks

In [ ]:
df_train.shape

(6753, 9)

In [ ]:
df_val.shape

(84, 9)

Shuffling the datasets of training and validation.

In [ ]:
df_train = df_train.sample(frac=1).reset_index(drop=True) #shuffle rows
df_val = df_val.sample(frac=1).reset_index(drop=True) #shuffle rows

I added one feature that encodes the features of the 6 labels chosen before. It is useful for the training of the model, because summarizes 6 features into only one.

In [ ]:
df_train['one_hot_labels'] = list(df_train[label_cols].values)
df_train.head()

,Self-direction: action,Achievement,Security: personal,Security: societal,Benevolence: caring,Universalism: concern,Premise,Stance,Conclusion,one_hot_labels
0,0,0,1,0,1,1,Foster children are given a stable home enviro...,against,Foster care brings more harm than good,"[0, 0, 1, 0, 1, 1]"
1,0,0,1,1,0,0,fast food is unhealthy and causes obesity,in favor of,We should ban fast food,"[0, 0, 1, 1, 0, 0]"
2,0,0,0,1,0,1,we need to do away with what there is so there...,in favor of,We should fight for the abolition of nuclear w...,"[0, 0, 0, 1, 0, 1]"
3,0,0,1,0,1,0,Immigration is a class issue. At the same time...,in favor of,We do not need immigration from non-European o...,"[0, 0, 1, 0, 1, 0]"
4,0,1,0,1,0,0,space exploration is needed to further our nat...,in favor of,We should subsidize space exploration,"[0, 1, 0, 1, 0, 0]"


In [ ]:
df_val['one_hot_labels'] = list(df_val[label_cols].values)
df_val.head()

,Self-direction: action,Achievement,Security: personal,Security: societal,Benevolence: caring,Universalism: concern,Premise,Stance,Conclusion,one_hot_labels
0,0,0,0,0,1,0,"Putting all the blame on the game, demonizing ...",against,We should restrict minors from playing online ...,"[0, 0, 0, 0, 1, 0]"
1,1,0,1,0,0,0,Extremely serious information asymmetry. It is...,in favor of,We should protect our privacy in the Internet ...,"[1, 0, 1, 0, 0, 0]"
2,0,0,0,1,0,0,The total production is controllable and the v...,against,We should ban the trading of virtual currencies.,"[0, 0, 0, 1, 0, 0]"
3,0,1,0,0,0,0,The electric car is driven by an electric moto...,against,We should give up developing electric vehicles,"[0, 1, 0, 0, 0, 0]"
4,0,1,0,1,0,0,Monopolies can sometimes protect the public in...,against,We should prohibit capital monopoly.,"[0, 1, 0, 1, 0, 0]"


Save the labels for the training session.

In [ ]:
labels_train = list(df_train.one_hot_labels.values)
labels_val= list(df_val.one_hot_labels.values)

#Merging the text arguments as 'Premise'+' '+'Stance'+' '+'Conclusion' to give as input to the model.

In [ ]:
list_text_merged_train=[]
for i in range(len(df_train)):
   list_text_merged_train.append(df_train['Premise'][i]+' '+df_train['Stance'][i]+' '+df_train['Conclusion'][i])

In [ ]:
list_text_merged_val=[]
for i in range(len(df_val)):
  list_text_merged_val.append(df_val['Premise'][i]+' '+df_val['Stance'][i]+' '+df_val['Conclusion'][i])
 

Remove punctuation, make the sentences lower case and perform lemmatization. This step standardizes the input, so that the model is not confused by misleading characters.

In [ ]:
def pre_tokenization(list_text_merged):

  new_list_text=[]
  for sentence in list_text_merged:
    new_sentence=sentence.translate(str.maketrans('', '', string.punctuation))
    new_sentence=new_sentence.lower()
    tokenization = nltk.word_tokenize(new_sentence)
    new_sentence_=''
    for w in tokenization:
      new_sentence_+=wordnet_lemmatizer.lemmatize(w)+' '
    new_list_text.append(new_sentence_)
  return new_list_text
list_text_merged_train=pre_tokenization(list_text_merged_train)
list_text_merged_val=pre_tokenization(list_text_merged_val)

AVG length of each list of train\validation set


In [ ]:
print('avg length of train text: ' ,sum( map(len, list_text_merged_train) ) / len(list_text_merged_train))

avg length of train text:  179.67940174737154


In [ ]:
print('avg length of train text: ' ,sum( map(len, list_text_merged_val) ) / len(list_text_merged_val))

avg length of train text:  210.17857142857142


The max length of the tokenizer is set to 300 in order to deal with memory issue and at the same time avoid truncation of the input sequence, as it is more far greater than the average length.

In [ ]:
max_length = 300
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) # tokenizer

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_ac

Performing input Tokenization.

In [ ]:
encodings_train = tokenizer.batch_encode_plus(list_text_merged_train,max_length=max_length,pad_to_max_length=True) # tokenizer's encoding method
encodings_val = tokenizer.batch_encode_plus(list_text_merged_val,max_length=max_length,pad_to_max_length=True) # tokenizer's encoding method


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
input_ids_train = encodings_train['input_ids'] # tokenized and encoded sentences
token_type_ids_train = encodings_train['token_type_ids'] # token type ids
attention_masks_train = encodings_train['attention_mask'] # attention masks


input_ids_val = encodings_val['input_ids'] # tokenized and encoded sentences
token_type_ids_val = encodings_val['token_type_ids'] # token type ids
attention_masks_val = encodings_val['attention_mask'] # attention masks

Convert all of our data into torch tensors, the required datatype for our model

In [ ]:

train_inputs=input_ids_train
train_labels=labels_train
train_masks=attention_masks_train
train_token_types=token_type_ids_train


train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
train_token_types = torch.tensor(train_token_types)


val_inputs=input_ids_val
val_labels=labels_val
val_masks=attention_masks_val
val_token_types=token_type_ids_val

validation_inputs = torch.tensor(val_inputs)
validation_labels = torch.tensor(val_labels)
validation_masks = torch.tensor(val_masks)
validation_token_types = torch.tensor(val_token_types)


<ipython-input-37-8a6d35bd0b60>:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  train_labels = torch.tensor(train_labels)


The batch size is set to 32 so that the notebook can run the free version of Google Colab

In [ ]:
batch_size = 32
# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory, this will ensure less use of GPU memory.

train_data = TensorDataset(train_inputs, train_masks, train_labels, train_token_types)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels, validation_token_types)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
torch.save(train_dataloader,'train_data_loader')
torch.save(validation_dataloader,'validation_data_loader')

# Load model, and set of parameters

In [ ]:
# Load model, the pretrained model will include a single linear classification layer on top for classification. 
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
model.cuda()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vo

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

# Optimizer

In [ ]:
optimizer = AdamW(model.parameters(),lr=2e-5) 

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


#Training of the model

The model is trained for 3 epochs using binary crossentropy loss.

In [ ]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 2

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):

  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()

  # Tracking variables
  tr_loss = 0 #running loss
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()

    # # Forward pass for multiclass classification
    # outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    # loss = outputs[0]
    # logits = outputs[1]

    # Forward pass for multilabel classification
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]
    loss_func = BCEWithLogitsLoss() 
    loss = loss_func(logits.view(-1,num_labels),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
    # loss_func = BCELoss() 
    # loss = loss_func(torch.sigmoid(logits.view(-1,num_labels)),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
    train_loss_set.append(loss.item())    

    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    # scheduler.step()
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))

###############################################################################

  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Variables to gather full output
  logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

  # Predict
  for i, batch in enumerate(validation_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    with torch.no_grad():
      # Forward pass
      outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      b_logit_pred = outs[0]
      pred_label = torch.sigmoid(b_logit_pred)

      b_logit_pred = b_logit_pred.detach().cpu().numpy()
      pred_label = pred_label.to('cpu').numpy()
      b_labels = b_labels.to('cpu').numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)

# Flatten outputs
pred_labels = [item for sublist in pred_labels for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]
# Calculate Accuracy
threshold = 0.50
pred_bools = [pl>threshold for pl in pred_labels]
true_bools = [tl==1 for tl in true_labels]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Train loss: 0.5670510779293079


Epoch:  50%|█████     | 1/2 [05:24<05:24, 324.81s/it]

Train loss: 0.4654148698977704


Epoch: 100%|██████████| 2/2 [10:49<00:00, 324.60s/it]


In [ ]:
# Print and save classification report

print('Test F1 Accuracy: ', f1_score(true_bools, pred_bools,average='micro'))
print('Test Flat Accuracy: ', accuracy_score(true_bools, pred_bools),'\n')
clf_report = classification_report(true_bools,pred_bools,target_names=label_cols)
pickle.dump(clf_report, open('classification_report.txt','wb')) #save report
print(clf_report)

Test F1 Accuracy:  0.5692883895131086
Test Flat Accuracy:  0.2261904761904762 

                        precision    recall  f1-score   support

Self-direction: action       0.60      0.55      0.57        11
           Achievement       0.68      0.67      0.68        39
    Security: personal       0.47      0.70      0.56        30
    Security: societal       1.00      0.26      0.41        31
   Benevolence: caring       0.60      0.50      0.55        12
 Universalism: concern       0.75      0.43      0.55        21

             micro avg       0.62      0.53      0.57       144
             macro avg       0.68      0.52      0.55       144
          weighted avg       0.70      0.53      0.56       144
           samples avg       0.60      0.56      0.54       144



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Optimization of the threshold 

The optimization of threshold is made by searching for the best results in terms of f1-macro and f1-micro score.

In [ ]:
macro_thresholds = np.array(range(1,10))/10

f1_results, flat_acc_results = [], []
for th in macro_thresholds:
  pred_bools = [pl>th for pl in pred_labels]
  test_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')
  test_flat_accuracy = accuracy_score(true_bools, pred_bools)
  f1_results.append(test_f1_accuracy)
  flat_acc_results.append(test_flat_accuracy)

best_macro_th = macro_thresholds[np.argmax(f1_results)] #best macro threshold value

micro_thresholds = (np.array(range(10))/100)+best_macro_th #calculating micro threshold values

f1_results, flat_acc_results = [], []
for th in micro_thresholds:
  pred_bools = [pl>th for pl in pred_labels]
  test_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')
  test_flat_accuracy = accuracy_score(true_bools, pred_bools)
  f1_results.append(test_f1_accuracy)
  flat_acc_results.append(test_flat_accuracy)

best_f1_idx = np.argmax(f1_results) #best threshold value


best_pred_bools = [pl>micro_thresholds[best_f1_idx] for pl in pred_labels]
clf_report_optimized = classification_report(true_bools,best_pred_bools, target_names=label_cols)
pickle.dump(clf_report_optimized, open('classification_report_optimized.txt','wb'))
print(clf_report_optimized)

Best Threshold:  0.44
Test F1 Accuracy:  0.5918367346938774
Test Flat Accuracy:  0.2261904761904762 

                        precision    recall  f1-score   support

Self-direction: action       0.53      0.73      0.62        11
           Achievement       0.67      0.74      0.71        39
    Security: personal       0.44      0.77      0.56        30
    Security: societal       0.91      0.32      0.48        31
   Benevolence: caring       0.44      0.58      0.50        12
 Universalism: concern       0.77      0.48      0.59        21

             micro avg       0.58      0.60      0.59       144
             macro avg       0.63      0.60      0.57       144
          weighted avg       0.66      0.60      0.59       144
           samples avg       0.60      0.63      0.58       144



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


reference:  https://towardsdatascience.com/transformers-for-multilabel-classification-71a1a0daf5e1